## Import packages

In [1]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

### csv 讀取成 pandas

In [2]:
import pandas as pd
df = pd.read_csv('covid.train.csv', index_col = 0)   # index_col = 0: 把第一個 column 當成 index
df

,AL,AK,AZ,AR,CA,CO,CT,FL,GA,ID,...,work_outside_home.4,shop.4,restaurant.4,spent_time.4,large_event.4,public_transit.4,anxious.4,depressed.4,worried_finances.4,tested_positive.4
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,1,0,0,...,31.113209,67.394551,36.674291,40.743132,17.842221,4.093712,10.440071,8.627117,37.329512,7.456154
1,0,0,0,0,0,1,0,0,0,0,...,33.920257,64.398380,34.612238,44.035688,17.808103,4.924935,10.172662,9.954333,32.508881,8.010957
2,0,0,0,0,0,0,0,0,0,0,...,31.604604,62.101064,26.521875,36.746453,13.903667,7.313833,10.388712,7.956139,36.745588,2.906977
3,0,0,0,0,0,0,0,0,0,0,...,35.115738,67.935520,38.022492,48.434809,27.134876,3.101904,10.498683,8.231522,38.680162,12.575816
4,0,0,0,0,0,0,0,0,0,1,...,35.129714,69.934592,38.242368,49.095933,22.683709,4.594620,9.878927,9.469290,28.344123,21.428589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,0,0,0,0,0,0,0,0,0,0,...,39.209278,65.815096,37.046827,46.777637,21.952499,2.724187,14.658436,11.099308,33.154910,13.437482
2696,0,0,0,0,0,0,0,0,0,0,...,28.984491,66.126427,31.299807,40.399527,16.783327,14.058799,13.129031,10.492896,36.670282,5.040143
2697,0,0,0,1,0,0,0,0,0,0,...,34.346100,63.456270,30.829009,41.828908,20.597714,2.391852,16.940896,12.888275,42.452451,13.618256


In [3]:
df.columns

Index(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'FL', 'GA', 'ID',
       ...
       'work_outside_home.4', 'shop.4', 'restaurant.4', 'spent_time.4',
       'large_event.4', 'public_transit.4', 'anxious.4', 'depressed.4',
       'worried_finances.4', 'tested_positive.4'],
      dtype='object', length=117)

In [4]:
df.columns.get_loc("cli.4")

101

In [30]:
feature_idx = [ df.columns.get_loc(c) for c in df.columns if "cli" in c or "ili" in c ]
df.columns[feature_idx]

Index(['cli', 'ili', 'hh_cmnty_cli', 'nohh_cmnty_cli', 'cli.1', 'ili.1',
       'hh_cmnty_cli.1', 'nohh_cmnty_cli.1', 'cli.2', 'ili.2',
       'hh_cmnty_cli.2', 'nohh_cmnty_cli.2', 'cli.3', 'ili.3',
       'hh_cmnty_cli.3', 'nohh_cmnty_cli.3', 'cli.4', 'ili.4',
       'hh_cmnty_cli.4', 'nohh_cmnty_cli.4'],
      dtype='object')

In [31]:
df.iloc[ : , feature_idx ]

,cli,ili,hh_cmnty_cli,nohh_cmnty_cli,cli.1,ili.1,hh_cmnty_cli.1,nohh_cmnty_cli.1,cli.2,ili.2,hh_cmnty_cli.2,nohh_cmnty_cli.2,cli.3,ili.3,hh_cmnty_cli.3,nohh_cmnty_cli.3,cli.4,ili.4,hh_cmnty_cli.4,nohh_cmnty_cli.4
id,,,,,,,,,,,,,,,,,,,,
0,0.658466,0.724606,12.939284,8.556230,0.653157,0.713249,12.488933,8.219380,0.677783,0.737965,12.487637,8.135510,0.666751,0.723506,12.367718,8.006131,0.685628,0.740943,12.364307,8.151578
1,0.693287,0.675962,15.008467,11.091818,0.738029,0.720511,15.070049,10.990937,0.831586,0.827523,14.568504,10.578924,0.767643,0.763580,14.264862,10.212529,0.713255,0.719378,12.894363,8.919288
2,0.617041,0.617041,8.614719,5.238097,0.663597,0.635459,8.472831,5.154184,0.534394,0.497435,8.493353,5.125553,0.536224,0.498305,8.227657,5.298912,0.479111,0.432445,8.067909,5.333533
3,1.193768,1.272051,12.007656,7.219741,1.356577,1.432775,12.997184,7.843776,1.719706,1.690806,14.010509,9.008978,1.737550,1.709244,14.364735,9.096583,1.793346,1.766201,14.383789,9.416096
4,1.455489,1.455489,20.957729,16.829163,1.298511,1.231606,20.189998,16.229791,1.486851,1.420611,20.160351,15.672487,1.373938,1.308346,20.103385,15.798003,1.264465,1.198567,20.257355,15.864201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,1.145121,1.147415,23.482648,19.457059,1.171500,1.163963,23.816931,19.707350,1.196115,1.163842,24.402762,20.106880,1.235034,1.212729,25.647724,21.239042,1.417447,1.401417,26.355591,21.616363
2696,0.693043,0.698368,11.641532,7.894431,0.655551,0.660840,11.519438,7.919253,0.635128,0.635128,11.514569,8.035914,0.563209,0.575010,11.500600,8.295961,0.605297,0.608920,11.135534,8.133172
2697,1.814052,1.771010,29.547019,24.115854,1.728777,1.721787,29.207283,23.868097,1.697214,1.750682,28.339583,23.089998,1.562703,1.617447,27.920069,22.261879,1.477089,1.566401,27.571493,21.874254
